In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings('ignore') #ignore warnings
import sklearn.exceptions
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-aug-2021/train.csv") #load train.csv
test = pd.read_csv("../input/tabular-playground-series-aug-2021/test.csv") #load test.csv
sample = pd.read_csv("../input/tabular-playground-series-aug-2021/sample_submission.csv") #load sample_submission.csv

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train.head(10)

In [ ]:
test.head(10)

In [ ]:
sample.head() #type of data need to submit

In [ ]:
#for i in [train, test]:
#    i.drop('id', axis = 1, inplace = True)

In [ ]:
#x = train.drop("loss", axis=1) #feature
y = train["loss"] #traget

In [ ]:
train.drop(['id', 'loss'], axis=1, inplace=True)
test.drop(['id'], axis=1, inplace=True)

In [ ]:
submit_types = ['id', 'loss']
features = []
for feat in train.columns:
    if feat not in submit_types:
        features.append(feat)
print(features)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train[features] = sc.fit_transform(train[features])
test[features] = sc.fit_transform(test[features])

In [ ]:
x = train #feature

In [ ]:
x

In [ ]:
y

In [ ]:
import lightgbm as lgbm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import optuna

In [ ]:
def find_params(trial, X = x, Y = y):
    x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size = 0.3,random_state = 42)
    lgbm_param = {
        "metric" : "rmse",
        "device_type" : "gpu",
        "boosting_type" : "gbdt", #Gradient Boosted Decision Tress
        "learning_rate" : trial.suggest_uniform("learning_rate", 0.02, 0.1),
        "lambda_l1" : trial.suggest_float("lambda_l1", 0.3, 0.5), #default=0
        "lambda_l2" : trial.suggest_float("lambda_l2", 0.3, 0.5), #default=0
        "num_leaves" : trial.suggest_int("num_leaves", 50, 100), #Number of leaves in the entire tree
        "max_depth" : trial.suggest_int("max_depth", 30, 40), #tree depth
        "n_estimators" : trial.suggest_int("n_estimators", 5000, 6000),
        "min_child_samples" : trial.suggest_int("min_child_samples", 50, 100),
        "colsample_bytree" : trial.suggest_int("colsample_bytree", 0.5, 0.7),
        "subsample" : trial.suggest_uniform("subsample", 0.8, 1),
        "feature_fraction" : trial.suggest_float("feature_fraction", 0.4, 1.0), #If its minimum value is too small, an error occurs
        "bagging_fraction" : trial.suggest_float("bagging_fraction", 0.4, 1.0) #If its minimum value is too small, an error occurs
    }
    model = lgbm.LGBMRegressor(**lgbm_param)
#    model.fit(x_train, y_train, eval_set = [(x_val, y_val)], early_stopping_rounds=100, verbose=False)
    model.fit(x_train, y_train, eval_set = [(x_val, y_val)], verbose = False)
    
    y_preds = model.predict(x_val)
    loss = np.sqrt(mean_squared_error(y_val, y_preds))
    
    return loss

In [ ]:
#train_time = 1 * 10 * 60 # Train for up to ten minutes
study = optuna.create_study(direction="minimize", study_name="lgbm")
study.optimize(find_params, n_trials=50)
#study.optimize(find_params, timeout=train_time) #this is not a very good option
print("Trial finished num: :", len(study.trials))
print("Best trial: score {}, params {}".format(study.best_trial.value, study.best_trial.params))

In [ ]:
display(optuna.visualization.plot_optimization_history(study))

In [ ]:
study.best_trial.params

In [ ]:
from sklearn.model_selection import StratifiedKFold, KFold, LeaveOneGroupOut
from lightgbm import LGBMRegressor

lgbm_best_params = study.best_trial.params
lgbm_best_params["metric"] = "RMSE"
lgbm_best_params["boosting_type"] = "gbdt"
lgbm_best_params["random_state"] = 42

folds = KFold(n_splits = 10, random_state = 42, shuffle = True)
predictions = None

for fold, (trn_idx, val_idx) in enumerate(folds.split(x.values, y.values)):
    print(f"Fold {fold+1}")
    x_train, x_val = x.values[trn_idx], x.values[val_idx]
    y_train, y_val = y.values[trn_idx], y.values[val_idx]
        
    model = LGBMRegressor(**lgbm_best_params, device="gpu")
#    model.fit(x_train, y_train, eval_set = [(x_val, y_val)], early_stopping_rounds=100, verbose=False)
    model.fit(x_train, y_train, eval_set = [(x_val, y_val)])   
    
    train_preds = model.predict(x_train)
    val_preds = model.predict(x_val)
    print(np.sqrt(mean_squared_error(y_val, val_preds)))
    
    if predictions is None: 
        predictions = model.predict(test.values)
    else:
        predictions += model.predict(test.values)

predictions/=10 #number of fold for CV. folds.n_splits

In [ ]:
sample['loss'] = predictions
sample.to_csv('lgbm_sub.csv', index = False)

In [ ]:
from xgboost import XGBRegressor
import xgboost as xgb

def find_params2(trial, X = x, Y = y):
    x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size = 0.3,random_state = 228)
    xgb_param = {
        "metric" : "rmse",
        "device_type" : "gpu",
        "tree_method" : "gpu_hist",
        "booster" : "gbtree",
        "learning_rate" : trial.suggest_uniform("learning_rate", 0.01, 0.1),
        "reg_alpha" : trial.suggest_float("reg_alpha", 0.005, 0.006), #default=0
        "reg_lambda" : trial.suggest_float("reg_lambda", 0.0001, 0.001), #default=1
        "max_depth" : trial.suggest_int("max_depth", 9, 11), #tree depth
        "n_estimators" : trial.suggest_int("n_estimators", 2000, 2500),
        "colsample_bytree" : trial.suggest_int("colsample_bytree", 0.5, 0.7), 
    }
    model = xgb.XGBRegressor(**xgb_param, predictor='gpu_predictor')
    model.fit(x_train, y_train, eval_set=[(x_val, y_val)], verbose = False)
    
    y_preds = model.predict(x_val)
    loss = np.sqrt(mean_squared_error(y_val, y_preds))
    
    return loss

In [ ]:
study2 = optuna.create_study(direction="minimize", study_name="xgb")
study2.optimize(find_params2, n_trials=50)
#study.optimize(find_params, timeout=train_time)
print("Trial finished num: :", len(study2.trials))
print("Best trial: score {}, params {}".format(study2.best_trial.value, study2.best_trial.params))

In [ ]:
display(optuna.visualization.plot_optimization_history(study2))

In [ ]:
study2.best_trial.params

In [ ]:
xgb_best_params = study2.best_trial.params
xgb_best_params["tree_method"] = "gpu_hist" #use gpu
xgb_best_params["random_state"] = 42

folds = KFold(n_splits = 10, random_state = 42, shuffle = True)
predictions = None

for fold, (trn_idx, val_idx) in enumerate(folds.split(x.values, y.values)):
    print(f"Fold {fold+1}")
    x_train, x_val = x.values[trn_idx], x.values[val_idx]
    y_train, y_val = y.values[trn_idx], y.values[val_idx]
    
    eval_set = [(x_val, y_val)]
    
    model = XGBRegressor(**xgb_best_params, device="gpu")
    model.fit(x_train, y_train, eval_set = eval_set)
    
    train_preds = model.predict(x_train)
    val_preds = model.predict(x_val)
    print(np.sqrt(mean_squared_error(y_val, val_preds)))
    
    if predictions is None: 
        predictions = model.predict(test.values)
    else:
        predictions += model.predict(test.values)

predictions/=10 #number of fold for CV. folds.n_splits 

In [ ]:
sample['loss'] = predictions
sample.to_csv('xgb_sub.csv', index = False)